In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os


import torch
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision import models

device = 'cuda' if torch.cuda.is_available() else 'cpu'

C:\Users\alireza\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\alireza\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
C:\Users\alireza\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
DATA_DIR = r'D:\Datasets\plant-seedlings-classification\train'

batch_size = 32
sz = 224

In [4]:
trfs = transforms.Compose([
    transforms.Resize(size=(sz, sz)),
    transforms.ToTensor()
]) 

In [5]:
whole_dataset = datasets.ImageFolder(DATA_DIR, transform=trfs)

In [6]:
whole_dataset[0]

(tensor([[[0.3137, 0.2863, 0.3098,  ..., 0.3725, 0.3804, 0.3451],
          [0.3059, 0.3020, 0.3255,  ..., 0.3647, 0.3569, 0.3333],
          [0.3059, 0.3137, 0.3333,  ..., 0.3608, 0.3490, 0.3451],
          ...,
          [0.5529, 0.5647, 0.5608,  ..., 0.3294, 0.2549, 0.2275],
          [0.5569, 0.5608, 0.5569,  ..., 0.3176, 0.2706, 0.2392],
          [0.5529, 0.5608, 0.5529,  ..., 0.3255, 0.2627, 0.2275]],
 
         [[0.1961, 0.1725, 0.2039,  ..., 0.3451, 0.3569, 0.3216],
          [0.1804, 0.1804, 0.2118,  ..., 0.3333, 0.3255, 0.2980],
          [0.1765, 0.1882, 0.2196,  ..., 0.3176, 0.3059, 0.3020],
          ...,
          [0.5804, 0.5922, 0.5882,  ..., 0.3255, 0.2353, 0.2078],
          [0.5843, 0.5882, 0.5843,  ..., 0.3176, 0.2549, 0.2235],
          [0.5843, 0.5922, 0.5882,  ..., 0.3255, 0.2510, 0.2118]],
 
         [[0.1059, 0.0784, 0.1176,  ..., 0.2353, 0.2392, 0.1882],
          [0.0824, 0.0863, 0.1294,  ..., 0.2431, 0.2275, 0.1922],
          [0.0784, 0.0941, 0.1373,  ...,

In [7]:
whole_dataset.imgs[0][0]

'D:\\Datasets\\plant-seedlings-classification\\train\\Black-grass\\0050f38b3.png'

In [8]:
len(whole_dataset)

4750

In [9]:
class_names = whole_dataset.classes

In [10]:
def array_random_pick(array, pick_num):
    index = np.arange(len(array))
    pick = np.random.choice(len(array), pick_num, replace=False)
    unpick = np.equal(np.in1d(index, pick), False)
    return array[unpick], array[pick]

In [11]:
train_mask, valid_mask = array_random_pick(np.arange(len(whole_dataset)), 500)

print(len(train_mask), len(valid_mask))

4250 500


In [12]:
train_set = torch.utils.data.Subset(whole_dataset, train_mask)
valid_set = torch.utils.data.Subset(whole_dataset, valid_mask)

print(len(train_set), len(valid_set))
print(train_set[0][0].shape)
print(valid_set[0][0].shape)

4250 500
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])


In [13]:
train_set.dataset.transforms = transforms.Compose([
    transforms.Resize(size=(sz, sz)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    
])

In [14]:
train_set.dataset.transforms

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
    RandomHorizontalFlip(p=0.5)
    ColorJitter(brightness=(0.9, 1.1), contrast=(0.9, 1.1), saturation=(0.9, 1.1), hue=(-0.1, 0.1))
    RandomRotation(degrees=[-20.0, 20.0], interpolation=nearest, expand=False, fill=0)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)

In [15]:
train_dl = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_dl = torch.utils.data.DataLoader(valid_set, batch_size=batch_size)

In [16]:
imgs, labels = next(iter(train_dl))

In [17]:
imgs.shape

torch.Size([32, 3, 224, 224])

In [18]:
model = models.resnet50(weights=True)

C:\Users\alireza\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [23]:
for params in model.parameters():
    params.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, len(class_names))

model = model.to(device)

In [24]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.04, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=2, gamma=0.9)

In [25]:
from train import train_model

In [30]:
resnet_model = train_model(model, train_dl, valid_dl, loss_fn, optimizer, device, scheduler, num_epochs=10)

Epoch 1/10
----------
  train Loss: 0.15737 Acc: 0.94165
  valid Loss: 0.67055 Acc: 0.82200

Epoch 2/10
----------
  train Loss: 0.15389 Acc: 0.94259
  valid Loss: 0.66241 Acc: 0.83600

Epoch 3/10
----------
  train Loss: 0.15170 Acc: 0.94494
  valid Loss: 0.67098 Acc: 0.81800

Epoch 4/10
----------
  train Loss: 0.14382 Acc: 0.95059
  valid Loss: 0.70871 Acc: 0.80600

Epoch 5/10
----------
  train Loss: 0.13451 Acc: 0.95224
  valid Loss: 0.66613 Acc: 0.82000

Epoch 6/10
----------
  train Loss: 0.14377 Acc: 0.94753
  valid Loss: 0.71397 Acc: 0.81000

Epoch 7/10
----------
  train Loss: 0.14129 Acc: 0.95082
  valid Loss: 0.66660 Acc: 0.83000

Epoch 8/10
----------
  train Loss: 0.13763 Acc: 0.95224
  valid Loss: 0.64344 Acc: 0.82400

Epoch 9/10
----------
  train Loss: 0.13808 Acc: 0.95082
  valid Loss: 0.68099 Acc: 0.81800

Epoch 10/10
----------
  train Loss: 0.12837 Acc: 0.95624
  valid Loss: 0.70325 Acc: 0.83400

Training complete in 34m 3s
Best val Acc: 0.8360


In [27]:
from train import validate_model

validate_model(model, valid_dl, loss_fn, device)

  valid Loss: 1.00325 Acc: 0.77200


tensor(0.7720, device='cuda:0')